# `TohuNamespace`


[TOC]


The class `TohuNamespace` allows grouping together other tohu generators and treating them as a single unit (which is used to implement the functionality of custom generators).

## Initialisation and adding field generators

In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.tohu_namespace_NEW_3 import TohuNamespaceNEW3

In [2]:
g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW3()
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

Note that when a field generator is added to the tohu namesapce, internally a new spawn is created. We can verify this by checking that the generators are different, for example in the case of `g1` and the field generator `aa`:

In [3]:
print(f"g1: {g1}")
print(f"aa: {tohu_namespace.field_generators['aa']}")

assert tohu_namespace.field_generators['aa'] is not g1
assert not tohu_namespace.field_generators['aa'].is_clone_of(g1)

g1: <Integer (id=dd0316)>
aa: <Integer (id=0b12ac)>


There is a convenience method `add_field_generators_from_dict()` which allows passing a dictionary, and it will call `add_field_generator()` for any tohu generators found in this dictionary (while ignoring any other values).

In [4]:
dct = {
    "aa": Integer(100, 200),
    "bb": HashDigest(length=6),
    "some_string": "this string will not be added because it is not a tohu generator",
    "cc": FakerGenerator(method="name"),
    "answer": 42  # this number will also not be added because it is not a tohu generator
}

tohu_namespace = TohuNamespaceNEW3()
tohu_namespace.add_field_generators_from_dict(dct)

In [5]:
tohu_namespace.field_generators

{'aa': <Integer (id=4f8911)>,
 'bb': <HashDigest (id=505d5d)>,
 'cc': <FakerGenerator (id=692bec)>}

## ~Adding non-field generators~

In [6]:
from tohu.loop_variable_NEW_3 import LoopVariableNEW3

In [7]:
xx = LoopVariableNEW3("xx", values=[4, 5, 6])

tohu_namespace = TohuNamespaceNEW3()
#tohu_namespace.add_non_field_generator("xx", xx, is_externally_managed=True)

In [8]:
tohu_namespace.field_generators

{}

In [9]:
#tohu_namespace.all_generators

Note that in this case the generator which is internally stored in the tohu namespace is actually a clone of `xx`.

In [10]:
#assert tohu_namespace.all_generators["xx"].is_clone_of(xx)

## Setting the `tohu_items_cls` attribute

Let's create a new tohu namespace and add both field generators and non-field generators.

In [14]:
g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW3()
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

print(f"{tohu_namespace}")
print(tohu_namespace.field_names)

<TohuNamespaceNEW3
   'aa': <Integer (id=232bc5)>
   'bb': <HashDigest (id=a08d7b)>
   'cc': <FakerGenerator (id=be6629)>
>
('aa', 'bb', 'cc')


Initially the `tohu_items_cls` attribute refers to a non-existent tohu items class:

In [12]:
tohu_namespace.tohu_items_cls

<NonExistentTohuItemsClass>

Once all desired generators have been added to the tohu namespace, we can call `set_tohu_items_class`, which will automatically create a tohu items class with the same field names as the generators contained in the namespace.

In [13]:
tohu_namespace.set_tohu_items_class(name="Quux")

In [14]:
tohu_namespace.tohu_items_cls

tohu.tohu_items_class.Quux

This items class can then be used to create individual tohu items.

In [15]:
tohu_namespace.tohu_items_cls(aa=100, bb="910A97", cc="Kristen Wallace")

Quux(aa=100, bb='910A97', cc='Kristen Wallace')

## Resetting the custom generator and producing tohu items

In [16]:
tohu_namespace.field_generators

{'aa': <Integer (id=5ad8e5)>,
 'bb': <HashDigest (id=99683b)>,
 'cc': <FakerGenerator (id=74f518)>}

In [17]:
tohu_namespace.reset(seed=11111)

print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


In [35]:
import pytest
from tohu.tohu_namespace_NEW_3 import NonExistentTohuItemsClassError

tohu_namespace = TohuNamespaceNEW3()
tohu_namespace.add_field_generator("aa", Integer(100, 200))

with pytest.raises(NonExistentTohuItemsClassError, match="Please call `set_tohu_items_class\(\)` on the tohu namespace before generating items."):
    print(next(tohu_namespace))

## Adding generators with (internal) dependencies

In [18]:
from tohu.derived_generators import Apply

In [19]:
aa = Integer(1, 9)
bb = Apply(lambda x: x*11, aa)
cc = Apply(lambda x: x*101, bb)

In [20]:
assert bb.arg_gens[0].is_clone_of(aa)
assert cc.arg_gens[0].is_clone_of(bb)

In [21]:
tohu_namespace = TohuNamespaceNEW3()
tohu_namespace.add_field_generator("rr", aa)
tohu_namespace.add_field_generator("ss", aa)
tohu_namespace.add_field_generator("tt", bb)
tohu_namespace.add_field_generator("uu", cc)
tohu_namespace.add_field_generator("vv", cc)
tohu_namespace.set_tohu_items_class("Quux")

In [22]:
tohu_namespace.field_generators

{'rr': <Integer (id=0c7183)>,
 'ss': <Integer (id=2d6ac6)>,
 'tt': <Apply (id=6b6885)>,
 'uu': <Apply (id=2a2c6d)>,
 'vv': <Apply (id=58f59f)>}

Note that even though the generator `aa` is added to the namespace twice (first with the name `"rr"` and then with the name `"ss"`), `ss` actually ends up as a _clone_ of `rr` (this is to ensure that the tohu items produced by the namespace contain the correct values).

The following checks that this works as expected.

In [23]:
assert tohu_namespace.field_generators["ss"].is_clone_of(tohu_namespace.field_generators["rr"])
assert tohu_namespace.field_generators["tt"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["rr"])
assert tohu_namespace.field_generators["uu"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])
assert tohu_namespace.field_generators["vv"].parent.arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])

tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss=4, tt=88, uu=8888, vv=8888)
Quux(rr=9, ss=5, tt=99, uu=9999, vv=9999)
Quux(rr=6, ss=1, tt=66, uu=6666, vv=6666)
Quux(rr=6, ss=1, tt=66, uu=6666, vv=6666)
Quux(rr=3, ss=5, tt=33, uu=3333, vv=3333)


## Adding generators with external dependencies

In [24]:
xx = LoopVariableNEW3("xx", values=[111, 222, 333])
xx_spawned = xx.spawn()

dependency_mapping = {xx: xx_spawned}

In [25]:
aa = Integer(1, 9)
bb = Apply(lambda x, a: f"{x}-{11*a}", xx, aa)

In [26]:
assert bb.arg_gens[0].is_clone_of(xx)
assert bb.arg_gens[1].is_clone_of(aa)
#assert cc.arg_gens[0].is_clone_of(bb)

In [27]:
tohu_namespace = TohuNamespaceNEW3(dependency_mapping=dependency_mapping)
tohu_namespace.add_field_generator("rr", aa)
tohu_namespace.add_field_generator("ss", bb)
tohu_namespace.set_tohu_items_class("Quux")

In [28]:
# assert tohu_namespace.field_generators["ss"].arg_gens[0].is_clone_of(xx_spawned)
# assert tohu_namespace.field_generators["ss"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["rr"])
# #assert tohu_namespace.field_generators["vv"].parent.arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])

tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss='111-88')
Quux(rr=9, ss='111-99')
Quux(rr=6, ss='111-66')
Quux(rr=6, ss='111-66')
Quux(rr=3, ss='111-33')


Consequently, changing the state of `xx` does not have any effect on the values produced.

In [29]:
xx.update_current_value(333)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=6, ss='111-66')
Quux(rr=1, ss='111-11')
Quux(rr=6, ss='111-66')


By contrast, changing the state of `xx_spawned` does have an effect (as expected).

In [30]:
xx_spawned.update_current_value(333)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss='333-88')
Quux(rr=5, ss='333-55')
Quux(rr=7, ss='333-77')


Of course, resetting the tohu namespace does not affect the state of external dependencies.

In [31]:
tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss='333-88')
Quux(rr=9, ss='333-99')
Quux(rr=6, ss='333-66')
